In [1]:
%reset

In [2]:
import multiprocessing
import numpy as np
from heterogeneous_spatial_networks_funcs import (
    filepath_str,
    filename_str
)

In [3]:
# This may or may not correspond to the number of cpus for optimal
# parallelization performance. Feel free to modify if you see fit.
cpu_num = int(np.floor(multiprocessing.cpu_count()/2))

# Initialization of identification information for this particular batch
# of Delaunay-triangulated networks
network = "delaunay"
date = "20241016"
batch = "A"
scheme = "random"

network_str = f"network_{network}"
date_str = f"date_{date}"
batch_str = f"batch_{batch}"
scheme_str = f"scheme_{scheme}"

dim_str = "dim"
b_str = "b"
n_str = "n"
eta_n_str = "eta_n"
config_str = "config"

filepath = filepath_str(network)
filename_prefix = filepath + f"{date}{batch}"

identifier_filename = filename_prefix + "-identifier" + ".txt"
dim_filename = filename_prefix + f"-{dim_str}" + ".dat"
b_filename = filename_prefix + f"-{b_str}" + ".dat"
n_filename = filename_prefix + f"-{n_str}" + ".dat"
eta_n_filename = filename_prefix + f"-{eta_n_str}" + ".dat"
config_filename = filename_prefix + f"-{config_str}" + ".dat"
params_filename = filename_prefix + "-params" + ".dat"
sample_params_filename = filename_prefix + "-sample_params" + ".dat"
sample_config_params_filename = (
    filename_prefix + "-sample_config_params" + ".dat"
)

identifier_arr = np.asarray(
    [
        network_str,
        date_str,
        batch_str,
        scheme_str,
        dim_str,
        b_str,
        n_str,
        eta_n_str
    ]
)

# Initialization of fundamental parameters for Delaunay-triangulated
# networks
dim_arr = np.asarray([2, 3], dtype=int)
b_arr = np.asarray([1.0])
n_arr = np.asarray([1000], dtype=int)
eta_n_arr = np.asarray([0.0001, 0.001, 0.01, 0.1])
config_arr = np.arange(10, dtype=int) # 0, 1, 2, ..., 8, 9

dim_num = np.shape(dim_arr)[0]
b_num = np.shape(b_arr)[0]
n_num = np.shape(n_arr)[0]
eta_n_num = np.shape(eta_n_arr)[0]
config_num = np.shape(config_arr)[0]

sample_num = dim_num * b_num * n_num * eta_n_num
sample_config_num = sample_num * config_num

# Populate the network parameters array
params_arr = np.empty([sample_num, 4])
sample = 0
for dim in np.nditer(dim_arr):
    for b in np.nditer(b_arr):
        for n in np.nditer(n_arr):
            for eta_n in np.nditer(eta_n_arr):
                params_arr[sample, :] = (
                    np.asarray(
                        [
                            dim,
                            b,
                            n,
                            eta_n
                        ]
                    )
                )
                sample += 1

# Populate the network sample parameters array
sample_params_arr = np.empty([sample_num, 5])
sample = 0
for dim in np.nditer(dim_arr):
    for b in np.nditer(b_arr):
        for n in np.nditer(n_arr):
            for eta_n in np.nditer(eta_n_arr):
                sample_params_arr[sample, :] = (
                    np.asarray(
                        [
                            sample,
                            dim,
                            b,
                            n,
                            eta_n
                        ]
                    )
                )
                sample += 1

# Populate the network sample configuration parameters array
sample_config_params_arr = np.empty([sample_config_num, 6])
sample = 0
indx = 0
for dim in np.nditer(dim_arr):
    for b in np.nditer(b_arr):
        for n in np.nditer(n_arr):
            for eta_n in np.nditer(eta_n_arr):
                for config in np.nditer(config_arr):
                    sample_config_params_arr[indx, :] = (
                        np.asarray(
                            [
                                sample,
                                dim,
                                b,
                                n,
                                eta_n,
                                config
                            ]
                        )
                    )
                    indx += 1
                sample += 1

# Save identification information and fundamental network parameters
np.savetxt(identifier_filename, identifier_arr, fmt="%s")
np.savetxt(dim_filename, dim_arr, fmt="%d")
np.savetxt(b_filename, b_arr)
np.savetxt(n_filename, n_arr, fmt="%d")
np.savetxt(eta_n_filename, eta_n_arr)
np.savetxt(config_filename, config_arr, fmt="%d")
np.savetxt(params_filename, params_arr)
np.savetxt(sample_params_filename, sample_params_arr)
np.savetxt(sample_config_params_filename, sample_config_params_arr)

In [4]:
# Calculate and save L for each Delaunay-triangulated network parameter
# sample
from heterogeneous_spatial_networks_worker import run_delaunay_L

delaunay_L_params_list = list(map(tuple, sample_params_arr))
delaunay_L_args = (
    [
        (network, date, batch, int(sample), int(dim), b, int(n), eta_n)
        for (sample, dim, b, n, eta_n) in delaunay_L_params_list
    ]
)

__spec__ = None
with multiprocessing.Pool(processes=cpu_num) as pool:
    pool.map(run_delaunay_L, delaunay_L_args)

In [5]:
# Perform the node seeding procedure for each Delaunay-triangulated
# network parameter sample
from heterogeneous_spatial_networks_worker import run_node_seeding

max_try = 500

node_seeding_params_arr = (
    sample_config_params_arr[:, [0, 1, 2, 3, 5]]
) # sample, dim, b, n, config
node_seeding_params_list = list(map(tuple, node_seeding_params_arr))
node_seeding_args = (
    [
        (network, date, batch, int(sample), scheme, int(dim), b, int(n), int(config), int(max_try))
        for (sample, dim, b, n, config) in node_seeding_params_list
    ]
)

__spec__ = None
with multiprocessing.Pool(processes=cpu_num) as pool:
    pool.map(run_node_seeding, node_seeding_args)

In [6]:
# Check to see if the number of seeded nodes, prhd_n, equals the
# intended/specified number of nodes to be seeded, n. Continue to the
# topology initialization procedure ONLY IF prhd_n = n. If prhd_n != n
# for any specified network, then the code block identifies which
# particular set(s) of network parameters prhd_n != n occurred for.
if scheme == "prhd":
    prhd_n_vs_n = np.zeros(sample_config_num)
    for indx in range(sample_config_num):
        sample = int(sample_config_params_arr[indx, 0])
        n = int(sample_config_params_arr[indx, 3])
        config = int(sample_config_params_arr[indx, 5])
        
        config_filename = (
            filename_str(network, date, batch, sample)
            + f"C{config:d}" + ".config"
        )
        coords = np.loadtxt(config_filename)
        
        if np.shape(coords)[0] == n: prhd_n_vs_n[indx] = 1
        else: pass

    sample_config_params_prhd_n_neq_n = (
        sample_config_params_arr[np.where(prhd_n_vs_n == 0)]
    )
    
    if np.shape(sample_config_params_prhd_n_neq_n)[0] == 0:
        print_str = "Success! prhd_n = n for all network parameters!"
        print(print_str)
    elif np.shape(sample_config_params_prhd_n_neq_n)[0] > 0:
        print_str = (
            "prhd_n != n for at least one set of network parameters. "
            + "Repeat the periodic random hard disk node placement "
            + "procedure for the applicable set of network parameters before "
            + "continuing on to the topology initialization procedure."
        )
        print(print_str)

In [7]:
# Perform the topology initialization procedure for each
# Delaunay-triangulated network parameter sample
from heterogeneous_spatial_networks_worker import (
    run_delaunay_network_topology_initialization
)

delaunay_network_topology_initialization_params_arr = (
    sample_config_params_arr[:, [0, 1, 3, 5]]
) # sample, dim, n, config
delaunay_network_topology_initialization_params_list = (
    list(map(tuple, delaunay_network_topology_initialization_params_arr))
)
delaunay_network_topology_initialization_args = (
    [
        (network, date, batch, int(sample), scheme, int(dim), int(n), int(config))
        for (sample, dim, n, config) in delaunay_network_topology_initialization_params_list
    ]
)

__spec__ = None
with multiprocessing.Pool(processes=cpu_num) as pool:
    pool.map(
        run_delaunay_network_topology_initialization,
        delaunay_network_topology_initialization_args)